# Data here: 
## One and two day type 
## Three day type

# Day Type: One/two
## Create Date and Day of Week columns in the DF

In [34]:
#importing packages needed for analysis
import os
import numpy as np
import pandas as pd
import math
from pandas import DataFrame

path = os.getcwd()
#print(path)

#this code creates an output directory in the parent director, if one does not exist yet
#Note: this is where all of the output files will be written, since outputs are large this saves space in git
path = os.getcwd()
parent = os.path.dirname(path)
outputs_dir = parent+'\outputs'
if not os.path.exists(outputs_dir):
    os.makedirs(outputs_dir)
print('output files are written out in parent directory: '+outputs_dir)

load_dur1 = pd.read_csv('outputs/load_long_format.csv', index_col=0)
load_years = pd.read_csv('inputs/load_years.csv').dropna()
#print(load_dur1.head())
print(load_years)

output files are written out in parent directory: C:\Users\tgoforth\Documents\IPM temporal resolution project\outputs
           2016        2011
0    2016-01-01  2011-01-01
1    2016-01-02  2011-01-02
2    2016-01-03  2011-01-03
3    2016-01-04  2011-01-04
4    2016-01-05  2011-01-05
..          ...         ...
360  2016-12-26  2011-12-27
361  2016-12-27  2011-12-28
362  2016-12-28  2011-12-29
363  2016-12-29  2011-12-30
364  2016-12-30  2011-12-31

[365 rows x 2 columns]


In [13]:
from itertools import cycle

#repeat the date for 24 times for each hour in the day
year_2016 = load_years['2016'].repeat(24).reset_index(drop=True)
year_2011 = load_years['2011'].repeat(24).reset_index(drop=True)
#print(year_2016)

#use itertools.cycle to make the date repeat until it reaches the end of the dataframe
year_2016 = cycle(year_2016)
year_2011 = cycle(year_2011)

#create temporary data frames where they will iterate through. Needed to achieve the correct length of the DF
temp2016 = load_dur1.loc[load_dur1['R_Group'] == 'ERC'].copy()
temp2011 = load_dur1.loc[load_dur1['R_Group'] != 'ERC'].copy()

#iterate through the date until the end of DF 
temp2016['Date'] = [next(year_2016) for year in range(len(temp2016))]
temp2011['Date'] = [next(year_2011) for year in range(len(temp2011))]
#print(temp2016)
#print(temp2011)

load_dur2 = pd.concat([temp2016, temp2011], ignore_index=True)
#print(load_dur2)

#convert date to a datetime type 
load_dur2['Date'] = pd.to_datetime(load_dur2['Date'])
load_dur2['DOW'] = load_dur2['Date'].dt.weekday

#check if it is a weekday or not 
weekday = pd.read_csv('inputs/weekday.csv')
load_dur2 = pd.merge(load_dur2,weekday,on='DOW',how='left')
#print(load_dur2)

## Case 1: Monthly, single day type, 24 hours (288 segments)
#### Methodology: Using groupby function to group first by month, then by 24 hours

In [14]:
case1_load = load_dur2.copy()

aggregations = {'Load':['count',sum,'mean']}
case1 = case1_load.groupby(['Region','Month','Hour'],as_index=False).agg(aggregations)
case1.columns = case1.columns.droplevel(0)
case1.columns = ['Region','Month','Hour','Hour_Tot','Load_Tot','Load_Avg']
print(case1.head())
print('number of rows in dataset =',case1.shape[0])
#case1.to_csv('../outputs/load_segments_2daytype_monthly_24hr.csv')
print()

case1_load2 = pd.merge(case1_load,case1,on=['Region','Month','Hour'],how='left')
case1_load2 = case1_load2.sort_values(['Region','Load'])
print(case1_load2.head(3))
print('number of rows in dataset =',case1_load2.shape[0])
#case1_load2.to_csv('../outputs/load_8760_2daytype_monthly_24hr.csv')


     Region  Month  Hour  Hour_Tot  Load_Tot      Load_Avg
0  ERC_REST      1     1        31   1042446  33627.290323
1  ERC_REST      1     2        31   1051661  33924.548387
2  ERC_REST      1     3        31   1079739  34830.290323
3  ERC_REST      1     4        31   1144492  36919.096774
4  ERC_REST      1     5        31   1239385  39980.161290
number of rows in dataset = 18144

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
1545  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
823   ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1531  ERC_REST     ERC       REST  shoulder      3   72     3  27025   

           Date  DOW  Weekday  Hour_Tot  Load_Tot      Load_Avg  
1545 2016-03-05    5    False        31    886708  28603.483871  
823  2016-02-04    3     True        30    870591  29019.700000  
1531 2016-03-04    4     True        31    886708  28603.483871  
number of rows in dataset = 551880


## Case 2: Season, single day-type, 24 hours (72 segments)
#### Methodology: Use groupby function to group by season and hour

In [15]:
case2_load = load_dur2.copy()
case2_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case2_seasons = case2_seasons.drop(['bimonthly'], axis=1)
case2_seasons = case2_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case2_load = pd.merge(case2_load, case2_seasons, on='Month', how='left')
#print(case4_load)

aggregations = {'Load':['count',sum,'mean']}
case2 = case2_load.groupby(['Region','Season_Group','Hour'],as_index=False).agg(aggregations)
case2.columns = case2.columns.droplevel(0)
case2.columns = ['Region','Season_Group','Hour','Hour_Tot','Load_Tot','Load_Avg']
print(case2.head())
print('number of rows in dataset =',case2.shape[0])
case2.to_csv('../outputs/load_segments_2daytype_season_24hr.csv')
print()

case2_load2 = pd.merge(case2_load,case2,on=['Region','Season_Group','Hour'],how='left')
case2_load2 = case2_load2.sort_values(['Region','Load'])
print(case2_load2.head(3))
print('number of rows in dataset =',case2_load2.shape[0])
case2_load2.to_csv('../outputs/load_8760_2daytype_season_24hr.csv')

     Region  Season_Group  Hour  Hour_Tot  Load_Tot      Load_Avg
0  ERC_REST             1     1        90   2928629  32540.322222
1  ERC_REST             1     2        90   2929331  32548.122222
2  ERC_REST             1     3        90   2978696  33096.622222
3  ERC_REST             1     4        90   3116896  34632.177778
4  ERC_REST             1     5        90   3346197  37179.966667
number of rows in dataset = 7560

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
1545  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
823   ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1531  ERC_REST     ERC       REST  shoulder      3   72     3  27025   

           Date  DOW  Weekday  Season_Group  Hour_Tot  Load_Tot      Load_Avg  
1545 2016-03-05    5    False             2       122   3603145  29533.975410  
823  2016-02-04    3     True             2       122   3587428  29405.147541  
1531 2016-03-04    4     True            

## Case 3: Monthly, weekend/weekday, 24 hours (576 segments)
#### Metholodogy: Use groupby function to group by month, then weekend/weekday, then by 24 hours

In [16]:
case3_load = load_dur2.copy()

aggregations = {'Load':['count',sum,'mean']}
case3 = case3_load.groupby(['Region','Month','Weekday','Hour'],as_index=False).agg(aggregations)
case3.columns = case3.columns.droplevel(0)
case3.columns = ['Region','Month','Weekday','Hour','Hour_Tot','Load_Tot','Load_Avg']
print(case1.head())
print('number of rows in dataset =',case3.shape[0])
#case3.to_csv('../outputs/load_segments_2daytype_monthly_wkd.csv')
print()

case3_load2 = pd.merge(case3_load,case3,on=['Region','Month','Weekday','Hour'],how='left')
case3_load2 = case3_load2.sort_values(['Region','Load'])
print(case3_load2.head())
print('number of rows in dataset =',case3_load2.shape[0])
#case3_load2.to_csv('../outputs/load_8760_2daytype_monthly_wkd.csv')

     Region  Month  Hour  Hour_Tot  Load_Tot      Load_Avg
0  ERC_REST      1     1        31   1042446  33627.290323
1  ERC_REST      1     2        31   1051661  33924.548387
2  ERC_REST      1     3        31   1079739  34830.290323
3  ERC_REST      1     4        31   1144492  36919.096774
4  ERC_REST      1     5        31   1239385  39980.161290
number of rows in dataset = 24507

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
1545  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
823   ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1531  ERC_REST     ERC       REST  shoulder      3   72     3  27025   
1524  ERC_REST     ERC       REST  shoulder      3   65     3  27059   
794   ERC_REST     ERC       REST  shoulder      3   65     2  27069   

           Date  DOW  Weekday  Hour_Tot  Load_Tot      Load_Avg  
1545 2016-03-05    5    False        14    406333  29023.785714  
823  2016-02-04    3     True        30    870

## Case 4: Season, weekend/weekday, 24 hours (144 segments)
#### Methodology: groupby season, weekday, hour

In [17]:
case4_load = load_dur2.copy()
case4_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case4_seasons = case4_seasons.drop(['bimonthly'], axis=1)
case4_seasons = case4_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case4_load = pd.merge(case4_load, case4_seasons, on='Month', how='left')
#print(case4_load)

aggregations = {'Load':['count',sum,'mean']}
case4 = case4_load.groupby(['Region','Season_Group','Weekday','Hour'],as_index=False).agg(aggregations)
case4.columns = case4.columns.droplevel(0)
case4.columns = ['Region','Season_Group','Weekday','Hour','Hour_Tot','Load_Tot','Load_Avg']
print(case4.head())
print('number of rows in dataset =',case4.shape[0])
#case4.to_csv('../outputs/load_segments_2daytype_season_wkd.csv')
print()

case4_load2 = pd.merge(case4_load,case4,on=['Region','Season_Group','Weekday','Hour'],how='left')
case4_load2 = case4_load2.sort_values(['Region','Load'])
print(case4_load2.head())
print('number of rows in dataset =',case4_load2.shape[0])
#case4_load2.to_csv('../outputs/load_8760_2daytype_season_wkd.csv')

     Region  Season_Group  Weekday  Hour  Hour_Tot  Load_Tot      Load_Avg
0  ERC_REST             1    False     1        40   1226650  30666.250000
1  ERC_REST             1    False     2        30   1003087  33436.233333
2  ERC_REST             1    False     4        52   1741883  33497.750000
3  ERC_REST             1    False     5        18    669258  37181.000000
4  ERC_REST             1    False     7        64   2446079  38219.984375
number of rows in dataset = 12478

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
1545  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
823   ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1531  ERC_REST     ERC       REST  shoulder      3   72     3  27025   
1524  ERC_REST     ERC       REST  shoulder      3   65     3  27059   
794   ERC_REST     ERC       REST  shoulder      3   65     2  27069   

           Date  DOW  Weekday  Season_Group  Hour_Tot  Load_Tot      Load_Avg  
15

## 4 hour interval day types

In [24]:
#read in 4 hour interval counter
interval_4hr = pd.read_csv('inputs/sequential_hours.csv')
interval_4hr = interval_4hr.drop(columns=['2-hr','8-hr','12-hr','24-hr','48-hr','120-hr'])
print(interval_4hr)

#add an hour counter
load_dur2['Hour_Counter'] = (load_dur2['Hour']) + (load_dur2['Day'] - 1) * 24
load_dur2 = load_dur2.sort_values(by=['Region','Hour_Counter'])
unique_hc = pd.Series(load_dur2['Hour_Counter'].unique()).dropna()
#print(unique_hc.tail(2))

load_dur3 = pd.merge(load_dur2,interval_4hr,on='Hour_Counter',how='left')
print(load_dur3)

      Hour_Counter  4-hr
0                1     1
1                2     1
2                3     1
3                4     1
4                5     2
...            ...   ...
8755          8756  2189
8756          8757  2190
8757          8758  2190
8758          8759  2190
8759          8760  2190

[8760 rows x 2 columns]
          Region R_Group R_Subgroup  Season  Month  Day  Hour   Load  \
0       ERC_REST     ERC       REST  winter      1    1     1  34807   
1       ERC_REST     ERC       REST  winter      1    1     2  34551   
2       ERC_REST     ERC       REST  winter      1    1     3  34788   
3       ERC_REST     ERC       REST  winter      1    1     4  35531   
4       ERC_REST     ERC       REST  winter      1    1     5  36633   
...          ...     ...        ...     ...    ...  ...   ...    ...   
551875  WEC_SDGE     WEC       SDGE  winter     12  365    20   2809   
551876  WEC_SDGE     WEC       SDGE  winter     12  365    21   2675   
551877  WEC_SDGE     WEC   

## Case 5: Monthly, single day type, 4 hour intervals (72 segments)
#### Methodology: use groupby by month, 4 hour intervals

In [ ]:
case5_load = load_dur3.copy()

aggregations = {'Load':['count',sum,'mean']}
case5 = case5_load.groupby(['Region','Month','4-hr'],as_index=False).agg(aggregations)
case5.columns = case5.columns.droplevel(0)
case5.columns = ['Region','Month','4-hr','Hour_Tot','Load_Tot','Load_Avg']
print(case5.head())
print('number of rows in dataset =',case5.shape[0])
#case5.to_csv('../outputs/load_segments_2daytype_month_4hr.csv')
print()

case5_load2 = pd.merge(case5_load,case5,on=['Region','Month','4-hr'],how='left')
case5_load2 = case5_load2.sort_values(['Region','Load'])
print(case5_load2.head())
print('number of rows in dataset =',case5_load2.shape[0])
#case5_load2.to_csv('../outputs/load_8760_2daytype_month_4hr.csv')

## Case 6: Bi-monthly weekend/weekday day-types, 4-hour intervals
#### Methodology: use groupby function and bimonthly groups

In [ ]:
case6_load = load_dur3.copy()
case6_bimonth = pd.read_csv('inputs/season_bimonthly.csv')
case6_bimonth = case6_bimonth.drop(['seasonal'], axis=1)
case6_bimonth = case6_bimonth.rename(columns={'bimonthly':'Bimonth','month':'Month'})

case6_load = pd.merge(case6_load, case6_bimonth, on='Month', how='left')
#print(case4_load)

aggregations = {'Load':['count',sum,'mean']}
case6 = case6_load.groupby(['Region','Bimonth','4-hr'],as_index=False).agg(aggregations)
case6.columns = case6.columns.droplevel(0)
case6.columns = ['Region','Bimonth','4-hr','Hour_Tot','Load_Tot','Load_Avg']
print(case6.head())
print('number of rows in dataset =',case6.shape[0])
#case6.to_csv('../outputs/load_segments_2daytype_bimonth_4hr.csv')
print()

case6_load2 = pd.merge(case6_load,case6,on=['Region','Bimonth','4-hr'],how='left')
case6_load2 = case6_load2.sort_values(['Region','Load'])
print(case6_load2.head())
print('number of rows in dataset =',case6_load2.shape[0])
#case6_load2.to_csv('../outputs/load_8760_2daytype_bimonth_4hr.csv')

## Case 7: Season-based months, weekend/weekday day-types, 4-hour intervals
#### Methodology: groupby function and applied season groups

In [ ]:
case7_load = load_dur3.copy()
case7_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case7_seasons = case7_seasons.drop(['bimonthly'], axis=1)
case7_seasons = case7_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case7_load = pd.merge(case7_load, case7_seasons, on='Month', how='left')
#print(case4_load)

aggregations = {'Load':['count',sum,'mean']}
case7 = case7_load.groupby(['Region','Season','Weekday','4-hr'],as_index=False).agg(aggregations)
case7.columns = case7.columns.droplevel(0)
case7.columns = ['Region','Season','Weekday','4-hr','Hour_Tot','Load_Tot','Load_Avg']
print(case7.head())
print('number of rows in dataset =',case7.shape[0])
case7.to_csv('../outputs/load_segments_2daytype_season_wkd_4hr.csv')
print()

case7_load2 = pd.merge(case7_load,case7,on=['Region','Season','Weekday','4-hr'],how='left')
case7_load2 = case7_load2.sort_values(['Region','Load'])
print(case7_load2.head())
print('number of rows in dataset =',case7_load2.shape[0])
case7_load2.to_csv('../outputs/load_8760_2daytype_season_wkd_4hr.csv')

# Day type: Three (Weekday, Weekend, Peak Load)
## Find Peak Load Days in Each Month

In [25]:
#create temporary DF to find peak
test = pd.read_csv('outputs/load_long_format.csv')

#groupby region, month, and day to sum the total day load
aggregations1 = {'Load':sum}
test_sum = test.groupby(['Region','Month','Day'],as_index=False).agg(aggregations1)
#test1.columns = test1.columns.droplevel(0)
test_sum.columns = ['Region','Month','Day','Load_Tot']
#print(test_sum.head())
#print('number of rows in dataset =',test_sum.shape[0])

test3 = pd.merge(test,test_sum,on=['Region','Month','Day'],how='left')
#print(test3)

#groupby region and month to find maximum load
aggregations2 = {'Load_Tot':max}
test_max = test_sum.groupby(['Region','Month'],as_index=False).agg(aggregations2)
test_max.columns = ['Region','Month','Load_Max']
#print(test_max.head())

test4 = pd.merge(test3,test_max,on=['Region','Month'],how='left')
test4 = test4.drop(test4.columns[0], axis=1)
print(test4)

          Region R_Group R_Subgroup  Season  Month  Day  Hour   Load  \
0       ERC_REST     ERC       REST  winter      1    1     1  34807   
1       ERC_REST     ERC       REST  winter      1    2     1  34716   
2       ERC_REST     ERC       REST  winter      1    3     1  34736   
3       ERC_REST     ERC       REST  winter      1    4     1  35914   
4       ERC_REST     ERC       REST  winter      1    5     1  33845   
...          ...     ...        ...     ...    ...  ...   ...    ...   
551875   WECC_WY     WEC         WY  winter     12  361    24   1767   
551876   WECC_WY     WEC         WY  winter     12  362    24   1744   
551877   WECC_WY     WEC         WY  winter     12  363    24   1805   
551878   WECC_WY     WEC         WY  winter     12  364    24   1791   
551879   WECC_WY     WEC         WY  winter     12  365    24   1834   

        Load_Tot  Load_Max  
0         917588   1000605  
1         863132   1000605  
2         934513   1000605  
3         960576   

In [28]:
three_load = pd.read_csv('outputs/load_long_format.csv', index_col=0)
load_years = pd.read_csv('inputs/load_years.csv').dropna()
#print(load_dur1.head())
#print(load_years)

from itertools import cycle

#repeat the date for 24 times for each hour in the day
year_2016 = load_years['2016'].repeat(24).reset_index(drop=True)
year_2011 = load_years['2011'].repeat(24).reset_index(drop=True)
#print(year_2016)

#use itertools.cycle to make the date repeat until it reaches the end of the dataframe
year_2016 = cycle(year_2016)
year_2011 = cycle(year_2011)

#create temporary data frames where they will iterate through. Needed to achieve the correct length of the DF
temp2016 = three_load.loc[three_load['R_Group'] == 'ERC'].copy()
temp2011 = three_load.loc[three_load['R_Group'] != 'ERC'].copy()

#iterate through the date until the end of DF 
temp2016['Date'] = [next(year_2016) for year in range(len(temp2016))]
temp2011['Date'] = [next(year_2011) for year in range(len(temp2011))]
#print(temp2016)
#print(temp2011)

three_load2 = pd.concat([temp2016, temp2011], ignore_index=True)
#print(load_dur2)

#convert date to a datetime type 
three_load2['Date'] = pd.to_datetime(three_load2['Date'])
three_load2['DOW'] = three_load2['Date'].dt.weekday

#check if it is a weekday or not 
weekday = pd.read_csv('inputs/weekday.csv')
three_load2 = pd.merge(three_load2,weekday,on='DOW',how='left')
#print(load_dur2)

three_load3 = pd.merge(three_load2,test4,on=['Region','R_Group','R_Subgroup','Season','Month','Day',
                                         'Hour','Load'],how='left')
three_load3 = three_load3.rename(columns={'Weekday':'Day_Type'})
#print(three_load3)

#Return True if the load total equals the day identified as the max
three_load3.loc[three_load3['Day_Type'] == True, 'Day_Type'] = 'Weekday'
three_load3.loc[three_load3['Day_Type'] == False, 'Day_Type'] = 'Weekend'
three_load3.loc[three_load3['Load_Tot'] == three_load3['Load_Max'], 'Day_Type'] = 'Peak'
three_load3 = three_load3.drop(['Load_Tot','Load_Max'], axis=1)
print(three_load3)

          Region R_Group R_Subgroup  Season  Month  Day  Hour   Load  \
0       ERC_REST     ERC       REST  winter      1    1     1  34807   
1       ERC_REST     ERC       REST  winter      1    2     1  34716   
2       ERC_REST     ERC       REST  winter      1    3     1  34736   
3       ERC_REST     ERC       REST  winter      1    4     1  35914   
4       ERC_REST     ERC       REST  winter      1    5     1  33845   
...          ...     ...        ...     ...    ...  ...   ...    ...   
551875   WECC_WY     WEC         WY  winter     12  361    24   1767   
551876   WECC_WY     WEC         WY  winter     12  362    24   1744   
551877   WECC_WY     WEC         WY  winter     12  363    24   1805   
551878   WECC_WY     WEC         WY  winter     12  364    24   1791   
551879   WECC_WY     WEC         WY  winter     12  365    24   1834   

             Date  DOW Day_Type  
0      2016-01-01    4  Weekday  
1      2016-01-01    4  Weekday  
2      2016-01-01    4  Weekday  

## Case 1: Monthly, Weekend/weekday/peak day-types, 24 hours (864 segments)
#### Methodology: similar to two day type, just adding in peak day types to sort by

In [ ]:
case1_load = three_load3.copy()

aggregations = {'Load':['count',sum,'mean']}
case1 = case1_load.groupby(['Region','Month','Day_Type','Hour'],as_index=False).agg(aggregations)
case1.columns = case1.columns.droplevel(0)
case1.columns = ['Region','Month','Day_Type','Hour','Hour_Tot','Load_Tot','Load_Avg']
print(case1.head())
print('number of rows in dataset =',case1.shape[0])
case1.to_csv('../outputs/load_segments_3daytype_monthly_24hr.csv')
print()

case1_load2 = pd.merge(case1_load,case1,on=['Region','Month','Day_Type','Hour'],how='left')
case1_load2 = case1_load2.sort_values(['Region','Load'])
print(case1_load2.head(3))
print('number of rows in dataset =',case1_load2.shape[0])
case1_load2.to_csv('../outputs/load_8760_3daytype_monthly_24hr.csv')

## Case 2: Season, weekend/weekday/peak day-types, 24-hours (216 segments)

In [ ]:
case2_load = three_load3.copy()

aggregations = {'Load':['count',sum,'mean']}
case2 = case2_load.groupby(['Region','Season','Day_Type','Hour'],as_index=False).agg(aggregations)
case2.columns = case2.columns.droplevel(0)
case2.columns = ['Region','Season','Day_Type','Hour','Hour_Tot','Load_Tot','Load_Avg']
print(case2.head())
print('number of rows in dataset =',case2.shape[0])
case2.to_csv('../outputs/load_segments_3daytype_season_24hr.csv')
print()

case2_load2 = pd.merge(case2_load,case2,on=['Region','Season','Day_Type','Hour'],how='left')
case2_load2 = case2_load2.sort_values(['Region','Load'])
print(case1_load2.head(3))
print('number of rows in dataset =',case1_load2.shape[0])
case2_load2.to_csv('../outputs/load_8760_3daytype_season_24hr.csv')

## Case 3: Annual, weekend/weekday/peak day-types, 24-hours (72 segments)

In [ ]:
case3_load = three_load3.copy()

aggregations = {'Load':['count',sum,'mean']}
case3 = case3_load.groupby(['Region','Day_Type','Hour'],as_index=False).agg(aggregations)
case3.columns = case3.columns.droplevel(0)
case3.columns = ['Region','Day_Type','Hour','Hour_Tot','Load_Tot','Load_Avg']
print(case3.head())
print('number of rows in dataset =',case3.shape[0])
case3.to_csv('../outputs/load_segments_3daytype_annual_24hr.csv')
print()

case3_load2 = pd.merge(case3_load,case3,on=['Region','Day_Type','Hour'],how='left')
case3_load2 = case3_load2.sort_values(['Region','Load'])
print(case3_load2.head(3))
print('number of rows in dataset =',case3_load2.shape[0])
case3_load2.to_csv('../outputs/load_8760_3daytype_annual_24hr.csv')

## 4 hour interval

In [29]:
#read in 4 hour interval counter
interval_4hr = pd.read_csv('inputs/sequential_hours.csv')
interval_4hr = interval_4hr.drop(columns=['2-hr','8-hr','12-hr','24-hr','48-hr','120-hr'])
print(interval_4hr)

#add an hour counter
three_load3['Hour_Counter'] = (three_load3['Hour']) + (three_load3['Day'] - 1) * 24
three_load3 = three_load3.sort_values(by=['Region','Hour_Counter'])
unique_hc = pd.Series(three_load3['Hour_Counter'].unique()).dropna()
#print(unique_hc.tail(2))

three_load4 = pd.merge(three_load3,interval_4hr,on='Hour_Counter',how='left')
print(load_dur3)

      Hour_Counter  4-hr
0                1     1
1                2     1
2                3     1
3                4     1
4                5     2
...            ...   ...
8755          8756  2189
8756          8757  2190
8757          8758  2190
8758          8759  2190
8759          8760  2190

[8760 rows x 2 columns]
          Region R_Group R_Subgroup  Season  Month  Day  Hour   Load  \
0       ERC_REST     ERC       REST  winter      1    1     1  34807   
1       ERC_REST     ERC       REST  winter      1    1     2  34551   
2       ERC_REST     ERC       REST  winter      1    1     3  34788   
3       ERC_REST     ERC       REST  winter      1    1     4  35531   
4       ERC_REST     ERC       REST  winter      1    1     5  36633   
...          ...     ...        ...     ...    ...  ...   ...    ...   
551875  WEC_SDGE     WEC       SDGE  winter     12  365    20   2809   
551876  WEC_SDGE     WEC       SDGE  winter     12  365    21   2675   
551877  WEC_SDGE     WEC   

## Case 4: Bi-monthly, weekend/weekday/peak day-types, 4-hour intervals

In [31]:
case4_load = three_load4.copy()
case4_bimonth = pd.read_csv('inputs/season_bimonthly.csv')
case4_bimonth = case4_bimonth.drop(['seasonal'], axis=1)
case4_bimonth = case4_bimonth.rename(columns={'bimonthly':'Bimonth','month':'Month'})

case4_load = pd.merge(case4_load, case4_bimonth, on='Month', how='left')
print(case4_load)

aggregations = {'Load':['count',sum,'mean']}
case4 = case4_load.groupby(['Region','Bimonth','Day_Type','4-hr'],as_index=False).agg(aggregations)
case4.columns = case4.columns.droplevel(0)
case4.columns = ['Region','Bimonth','Day_Type','4-hr','Hour_Tot','Load_Tot','Load_Avg']
print(case4.head())
print('number of rows in dataset =',case4.shape[0])
#case4.to_csv('../outputs/load_segments_3daytype_bimonth_4hr.csv')
print()

case4_load2 = pd.merge(case4_load,case4,on=['Region','Bimonth','Day_Type','4-hr'],how='left')
case4_load2 = case4_load2.sort_values(['Region','Load'])
print(case4_load2.head())
print('number of rows in dataset =',case4_load2.shape[0])
#case4_load2.to_csv('../outputs/load_8760_3daytype_bimonth_4hr.csv')

          Region R_Group R_Subgroup  Season  Month  Day  Hour   Load  \
0       ERC_REST     ERC       REST  winter      1    1     1  34807   
1       ERC_REST     ERC       REST  winter      1    1     2  34551   
2       ERC_REST     ERC       REST  winter      1    1     3  34788   
3       ERC_REST     ERC       REST  winter      1    1     4  35531   
4       ERC_REST     ERC       REST  winter      1    1     5  36633   
...          ...     ...        ...     ...    ...  ...   ...    ...   
551875  WEC_SDGE     WEC       SDGE  winter     12  365    20   2809   
551876  WEC_SDGE     WEC       SDGE  winter     12  365    21   2675   
551877  WEC_SDGE     WEC       SDGE  winter     12  365    22   2524   
551878  WEC_SDGE     WEC       SDGE  winter     12  365    23   2362   
551879  WEC_SDGE     WEC       SDGE  winter     12  365    24   2206   

             Date  DOW Day_Type  Hour_Counter  4-hr  Bimonth  
0      2016-01-01    4  Weekday             1     1        1  
1      20

## Case 5: Season-based months, weekend/weekday/peak day-types, 4-hour intervals

In [32]:
case5_load = three_load4.copy()
case5_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case5_seasons = case5_seasons.drop(['bimonthly'], axis=1)
case5_seasons = case5_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case5_load = pd.merge(case5_load, case5_seasons, on='Month', how='left')
#print(case5_load)

aggregations = {'Load':['count',sum,'mean']}
case5 = case5_load.groupby(['Region','Season_Group','Day_Type','4-hr'],as_index=False).agg(aggregations)
case5.columns = case5.columns.droplevel(0)
case5.columns = ['Region','Season_Group','Day_Type','4-hr','Hour_Tot','Load_Tot','Load_Avg']
print(case5.head())
print('number of rows in dataset =',case5.shape[0])
#case5.to_csv('../outputs/load_segments_3daytype_seasonbased_4hr.csv')
print()

case5_load2 = pd.merge(case5_load,case5,on=['Region','Season_Group','Day_Type','4-hr'],how='left')
case5_load2 = case5_load2.sort_values(['Region','Load'])
print(case5_load2.head())
print('number of rows in dataset =',case5_load2.shape[0])
#case5_load2.to_csv('../outputs/load_8760_3daytype_seasonbased_4hr.csv')

     Region  Season_Group Day_Type  4-hr  Hour_Tot  Load_Tot  Load_Avg
0  ERC_REST             1     Peak    55         4    161100  40275.00
1  ERC_REST             1     Peak    56         4    187905  46976.25
2  ERC_REST             1     Peak    57         4    168812  42203.00
3  ERC_REST             1     Peak    58         4    155705  38926.25
4  ERC_REST             1     Peak    59         4    172847  43211.75
number of rows in dataset = 243586

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
2042  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
2233  ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1706  ERC_REST     ERC       REST  shoulder      3   72     3  27025   
1538  ERC_REST     ERC       REST  shoulder      3   65     3  27059   
1537  ERC_REST     ERC       REST  shoulder      3   65     2  27069   

           Date  DOW Day_Type  Hour_Counter  4-hr  Season_Group  Hour_Tot  \
2042 2016-03-05    5  Weeken

## Case 6: Season, weekend/weekday/peak day-types, 4-hour intervals

In [33]:
case6_load = three_load4.copy()

aggregations = {'Load':['count',sum,'mean']}
case6 = case6_load.groupby(['Region','Season','Day_Type','4-hr'],as_index=False).agg(aggregations)
case6.columns = case6.columns.droplevel(0)
case6.columns = ['Region','Season','Day_Type','4-hr','Hour_Tot','Load_Tot','Load_Avg']
print(case6.head())
print('number of rows in dataset =',case6.shape[0])
#case6.to_csv('../outputs/load_segments_3daytype_season_4hr.csv')
print()

case6_load2 = pd.merge(case6_load,case6,on=['Region','Season','Day_Type','4-hr'],how='left')
case6_load2 = case6_load2.sort_values(['Region','Load'])
print(case6_load2.head())
print('number of rows in dataset =',case6_load2.shape[0])
#case6_load2.to_csv('../outputs/load_8760_3daytype_season_4hr.csv')

     Region    Season Day_Type  4-hr  Hour_Tot  Load_Tot  Load_Avg
0  ERC_REST  shoulder     Peak   535         4    120877  30219.25
1  ERC_REST  shoulder     Peak   536         4    142982  35745.50
2  ERC_REST  shoulder     Peak   537         4    155141  38785.25
3  ERC_REST  shoulder     Peak   538         4    170205  42551.25
4  ERC_REST  shoulder     Peak   539         4    165142  41285.50
number of rows in dataset = 243586

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
2042  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
2233  ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1706  ERC_REST     ERC       REST  shoulder      3   72     3  27025   
1538  ERC_REST     ERC       REST  shoulder      3   65     3  27059   
1537  ERC_REST     ERC       REST  shoulder      3   65     2  27069   

           Date  DOW Day_Type  Hour_Counter  4-hr  Hour_Tot  Load_Tot  \
2042 2016-03-05    5  Weekend          2043   511       